In [22]:
#импорт необходимых библиотек
import pandas as pd
import numpy as np

#библиотеки для работы с датасетом
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

#используемые модели
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV

#метрик качества
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score

# Домашнее задание 3

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

2. при обучении моделей обязательно использовать кроссвалидацию

3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

4. сделать выводы о том, какая модель справилась с задачей лучше других

5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

**Решение**

Загрузка датасета

In [23]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделение датасета на тренировочную и тестовую выборки

In [24]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

Преобразование признаков:
    
- gender, cholesterol применим OHE-кодирование

- age, height, weight, ap_hi, ap_lo - standardScaler

- gluc, smoke, alco, active - оставим пока как есть

In [25]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Объединение трансформеров с помощью FeatureUnion

In [26]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Создание моделей классификации

Модель 1. Логистическая регрессия

In [27]:
lr = LogisticRegression(random_state = 42)

Модель 2. Случайный лес

In [28]:
# Ищем лучшие параметры для модели RandomForestClassifier
rf = RandomForestClassifier(random_state=1)

parameters = {'n_estimators': [100, 150, 200],
              'max_depth': np.arange(2, 10),
              'min_samples_leaf': np.arange(1, 10)}

grid_search = GridSearchCV(estimator=rf,
                           param_grid=parameters,
                           scoring='roc_auc',
                           cv=5,
                           n_jobs=-1)

grid_search.fit(feature_processing.fit_transform(X_train), y_train)

print(f'Лучшие параметры для RandomForestClassifier: \n {grid_search.best_params_}')

Лучшие параметры для RandomForestClassifier: 
 {'max_depth': 9, 'min_samples_leaf': 3, 'n_estimators': 150}


In [29]:
# Строим модель с лучшими параметрами
rf = RandomForestClassifier(random_state=1,
                            n_estimators=150,
                            max_depth=9,
                            min_samples_leaf =3)

Модель 3. LGBMClassifier

In [30]:
# Ищем лучшие параметры для модели LGBMClassifier
lgbm =  LGBMClassifier(random_state=1)

parameters = {'num_leaves': [10, 30, 40],
              'max_depth': np.arange(2, 10),
              'learning_rate': [0.1, 0.01, 0.005],
              'n_estimators': [100, 150, 200]}

grid_search = GridSearchCV(estimator=lgbm,
                           param_grid=parameters,
                           scoring='roc_auc',
                           cv=5,
                           n_jobs=-1)

grid_search.fit(feature_processing.fit_transform(X_train), y_train)

print(f'Лучшие параметры для LGBMClassifier: \n {grid_search.best_params_}')

Лучшие параметры для LGBMClassifier: 
 {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'num_leaves': 10}


In [31]:
# Строим модель с лучшими параметрами
lgbm =  LGBMClassifier(random_state=1,
                       num_leaves= 10,
                       max_depth= 4,
                       learning_rate = 0.1,
                       n_estimators = 100)

Модель 4. KNeighborsClassifier

In [32]:
# Ищем лучшие параметры для модели KNeighborsClassifier
knn =  KNeighborsClassifier()

parameters = {'n_neighbors': np.arange(2, 101)}

grid_search = GridSearchCV(estimator=knn,
                           param_grid=parameters,
                           scoring='roc_auc',
                           cv=5,
                           n_jobs=-1)

grid_search.fit(feature_processing.fit_transform(X_train), y_train)

print(f'Лучшие параметры для KNeighborsClassifier: \n {grid_search.best_params_}')

Лучшие параметры для KNeighborsClassifier: 
 {'n_neighbors': 39}


In [33]:
# Строим модель с лучшими параметрами
knn =  KNeighborsClassifier(n_neighbors = 39)

Обучение модели с настроенными параметрами (кроссвалидация) и получение метрик

In [34]:
#функция обучения модели и получения метрик качества

def get_model_metrics(model, feats, X_train, y_train, X_test, y_test):
    
    #создаем классификатор
    classifier = Pipeline([('features',feats),
                       ('classifier', model),])

    #кросс-валидация
    cv_scores = cross_val_score(classifier,
                            X_train, 
                            y_train, 
                            cv=KFold(n_splits=16, shuffle=True, random_state=1),
                            scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print('CV score for {} is {}+-{}'.format(model, cv_score, cv_score_std))

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)

    #делаем предсказание
    y_score = classifier.predict_proba(X_test)[:, 1]

    #считаем метрики
    b=1
    roc_auc = roc_auc_score(y_test, y_score)
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    
    return {'precision': precision[ix], 'recall': recall[ix], 'roc_auc': roc_auc, 'f_score': fscore[ix]}

In [35]:
#пример работы
metrics = get_model_metrics(lr, feats, X_train, y_train, X_test, y_test)
metrics

CV score for LogisticRegression(random_state=42) is 0.7867944769038742+-0.009555718966569003


{'precision': 0.6474307596402173,
 'recall': 0.8375576036866359,
 'roc_auc': 0.7840347790421852,
 'f_score': 0.7303229695112764}

Сводная таблица с метриками качества классификации по четырем моделям

In [36]:
models = [lr, rf, lgbm, knn]
models_col = ['LogisticRegression',  
              'RandomForestClassifier',
              'LGBMClassifier',
              'KNeighborsClassifier']
precision_score_col = []
recall_score_col = []
roc_auc_col = []
f_score_col = []

for model in models:
    metrics = get_model_metrics(model, feats, X_train, y_train, X_test, y_test)
    precision_score_col.append(metrics['precision'])
    recall_score_col.append(metrics['recall'])
    roc_auc_col.append(metrics['roc_auc'])
    f_score_col.append(metrics['f_score'])

df = pd.DataFrame({'model': models_col, 
                   'precision': precision_score_col, 
                   'recall': recall_score_col,
                   'f_score': f_score_col,
                   'roc_auc_score': roc_auc_col})
df

CV score for LogisticRegression(random_state=42) is 0.7867944769038742+-0.009555718966569003
CV score for RandomForestClassifier(max_depth=9, min_samples_leaf=3, n_estimators=150,
                       random_state=1) is 0.8012529828437567+-0.008696025604543477
CV score for LGBMClassifier(max_depth=4, num_leaves=10, random_state=1) is 0.8033458719822189+-0.008518247672537548
CV score for KNeighborsClassifier(n_neighbors=39) is 0.7299258630149079+-0.010612177874755586


,model,precision,recall,f_score,roc_auc_score
0,LogisticRegression,0.647431,0.837558,0.730323,0.784035
1,RandomForestClassifier,0.675167,0.817512,0.739552,0.801003
2,LGBMClassifier,0.692416,0.794124,0.739791,0.802775
3,KNeighborsClassifier,0.594274,0.860945,0.703176,0.728240


**Вывод:** лучше всех с поставленной задачей справилась LGBMClassifier (метрики f_score и roc_auc_score у нее выше, чем у остальных моделей).